In [ ]:
pip install category_encoders

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from category_encoders import TargetEncoder

Load & check data

In [4]:
train = pd.read_csv("/Users/aybikesahinoglu/Desktop/kaggle/playground-series-s5e11/train.csv")

train.dtypes

id                        int64
annual_income           float64
debt_to_income_ratio    float64
credit_score              int64
loan_amount             float64
interest_rate           float64
gender                   object
marital_status           object
education_level          object
employment_status        object
loan_purpose             object
grade_subgrade           object
loan_paid_back          float64
dtype: object

In [9]:
train["gender"].unique()

array(['Female', 'Male', 'Other'], dtype=object)

In [10]:
train["marital_status"].unique()

array(['Single', 'Married', 'Divorced', 'Widowed'], dtype=object)

In [11]:
train["education_level"].unique()

array(['High School', "Master's", "Bachelor's", 'PhD', 'Other'],
      dtype=object)

In [14]:
train["employment_status"].unique()

array(['Self-employed', 'Employed', 'Unemployed', 'Retired', 'Student'],
      dtype=object)

In [15]:
train["loan_purpose"].unique()

array(['Other', 'Debt consolidation', 'Home', 'Education', 'Vacation',
       'Car', 'Medical', 'Business'], dtype=object)

In [16]:
train["grade_subgrade"].unique()

array(['C3', 'D3', 'C5', 'F1', 'D1', 'D5', 'C2', 'C1', 'F5', 'D4', 'C4',
       'D2', 'E5', 'B1', 'B2', 'F4', 'A4', 'E1', 'F2', 'B4', 'E4', 'B3',
       'E3', 'B5', 'E2', 'F3', 'A5', 'A3', 'A1', 'A2'], dtype=object)

Encoding categorical data

In [5]:
X = train.drop(columns=['loan_paid_back'])
y = train['loan_paid_back'] 

In [ ]:
# Split already done
X_train = X
y_train = y

# Define encoder
te = TargetEncoder(
    cols=['education_level', 'employment_status', 'loan_purpose', 'grade_subgrade'],
    smoothing=10
)

# Prepare out-of-fold encoded training data
kf = KFold(n_splits=5, shuffle=True, random_state=42)
X_train_enc = X_train.copy()

for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr = y_train.iloc[train_idx]
    
    te.fit(X_tr, y_tr)
    X_val_enc = te.transform(X_val)
    
    X_train_enc.loc[X_val.index, te.cols] = X_val_enc[te.cols]

# Fit final encoder on full training data
te.fit(X_train, y_train)

# Transform test data (assuming you have X_test already)
X_test_enc = te.transform(X_test)